In [1]:
suppressMessages(library(Seurat))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(viridis))
suppressMessages(library(ggplot2))
options(warn=-1)

In [2]:
set.seed(42)
getwd()

[1] "/data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Endothelial/code"

In [3]:
df_anno <- read.csv("../../../visiumSpatial/metadata/sample_annotation.csv", header = TRUE)

patient_region_id <- df_anno$patient_region_id
patient_region_id <- stringr::str_replace_all(patient_region_id, "/", "_")

names(patient_region_id) <- df_anno$sample_id

In [4]:
if(!dir.exists("../data/visium/cell_states_mapping_with_annotation")){
    dir.create("../data/visium/cell_states_mapping_with_annotation")
}

In [5]:
for(sample in names(patient_region_id)){
        message(glue::glue("processing sample: {sample}"))

        # load visium object
        obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/{sample}.rds"))
    
        DefaultAssay(obj) <- "c2l_props"
        p1 <- SpatialFeaturePlot(obj, features = "Endo", max.cutoff = "q99") + ggtitle("c2l: proportion") +
    scale_fill_viridis()

         DefaultAssay(obj) <- "SCT"
        
        all.markers <- readRDS("../data/coembed/MarkerGenesAfterAnnotation/annotation.rds")
        
        # only keep genes that are found in spatial dta
        gene.use <- intersect(rownames(obj), rownames(all.markers))
        all.markers <- all.markers[gene.use, ]
        
        # get the marker genes for each cluster
        all.markers <- all.markers %>%
            subset(p_val_adj < 0.05) %>%
            subset(., select = c("cluster", "gene"))
        
        # convert dataframe to list for module score calculation
        all.markers <- split(all.markers, all.markers$cluster)
        
    for(cl in names(all.markers)){
           all.markers[[cl]] <- all.markers[[cl]]$gene 
        }
    
        # get module score
        obj <- AddModuleScore(obj, features = all.markers, assay = "SCT")
        
        df_module_score <- as.data.frame(obj@meta.data)%>%
             select(., contains("Cluster"))
        colnames(df_module_score) <- names(all.markers)
    
        #celltypes <- c("CD8", "CD4", "NK", "NK_T")
    celltypes <- names(all.markers)
    print(celltypes)
    
        # plot module score after normalization by proportion from cell2location
        c2l_score_prop <- obj@assays$c2l_props@data["Endo", ]
        df_module_score_prop <- df_module_score * c2l_score_prop
        df_module_score_prop[df_module_score_prop <0] <- 0
        obj <- AddMetaData(obj, metadata = df_module_score_prop)
        plotlist <- SpatialFeaturePlot(obj, features = celltypes, combine = FALSE,
                                       max.cutoff = "q99")
        plotlist <- lapply(1:length(plotlist), function(i){
           p <- plotlist[[i]] + theme(legend.title = element_blank()) +
            scale_fill_viridis() + ggtitle(celltypes[[i]])
           return(p)
        })
    
        ps1 <- patchwork::wrap_plots(plotlist, nrow = 2) +
            patchwork::plot_annotation(title = "Normalized by proportion")
        
        
        patient_region <- patient_region_id[sample]

        pdf(glue::glue("../data/visium/cell_states_mapping_with_annotation/{sample}_{patient_region}.pdf"), width = 20, height = 10)
        print(p1+ps1)
        dev.off()
    }

processing sample: AKK006_157771

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.



[1] "Capillary_Endo"   "Venous_Endo"      "Arterial_Endo"    "Endocardial_Endo"
[5] "Lymphatic_Endo"  


Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

processing sample: Visium_1_CK279

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.



[1] "Capillary_Endo"   "Venous_Endo"      "Arterial_Endo"    "Endocardial_Endo"
[5] "Lymphatic_Endo"  


Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

processing sample: Visium_3_CK281

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.



...

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

processing sample: AKK001_157785

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.



[1] "Capillary_Endo"   "Venous_Endo"      "Arterial_Endo"    "Endocardial_Endo"
[5] "Lymphatic_Endo"  


Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

processing sample: Visium_5_CK283

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.



[1] "Capillary_Endo"   "Venous_Endo"      "Arterial_Endo"    "Endocardial_Endo"
[5] "Lymphatic_Endo"  


Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

